# Simulation study - Inference of pharmacokinetic model in a virtual patient

## Measure plasma concentration in a virtual patient
Applied dose: daily oral dose of 2 mg, starting from day 3.

In [23]:
import numpy as np

import erlotinib as erlo

# Define predictive model
path = erlo.ModelLibrary().one_compartment_pk_model()
mechanistic_model = erlo.PharmacokineticModel(path)
mechanistic_model.set_administration(compartment='central', direct=False)
error_models = [erlo.ConstantAndMultiplicativeGaussianErrorModel()]
predictive_model = erlo.PredictiveModel(mechanistic_model, error_models)

# Define a virtual patient
parameters = [
    0,  # Initial drug amount in central compartment [mg]
    0,  # Initial drug amount in dose compartment [mg]
    0.8,     # Volume of distribution in central compartment [L]
    10,     # Absorption rate from dose to central compartment [1/d]
    3,  # Elimination rate from central compartment [1/d]
    0.04,  # Base noise
    0.1]   # Relative noise

# Define dosing regimen
predictive_model.set_dosing_regimen(dose=2, start=0)

# Take virtual measurements
seed = 42
times = np.linspace(0, 1, num=10)
n_samples = 1
data = predictive_model.sample(parameters, times, n_samples, seed)

### Visualise virtual measurements

In [24]:
# Create scatter plot
fig = erlo.plots.PDTimeSeriesPlot()
fig.add_data(data, biomarker='central.drug_concentration', meas_key='Sample')
fig.set_axis_labels(xlabel=r'$\text{Time in day}$', ylabel=r'$\text{Plasma conc. in ng/mL}$')

# Show figure
fig.show()

**Figure 1:** Visualisation of the measured tumour growth in a virtual patient.

## Build prior predictive model

In [25]:
import pints

# Define prior distribution
log_prior_initial_drug_amount = pints.UniformLogPrior(0, 0.001)  # Fixed to 0 later
log_prior_volume = pints.HalfCauchyLogPrior(location=0, scale=1)
log_prior_absorption_rate = pints.HalfCauchyLogPrior(location=1, scale=2)
log_prior_elimination_rate = pints.HalfCauchyLogPrior(location=0, scale=0.1)
log_prior_sigma_base = pints.HalfCauchyLogPrior(location=0, scale=1)
log_prior_sigma_rel = pints.HalfCauchyLogPrior(location=0, scale=0.1)
log_prior = pints.ComposedLogPrior(
    log_prior_initial_drug_amount, log_prior_initial_drug_amount, log_prior_volume, log_prior_absorption_rate,
    log_prior_elimination_rate, log_prior_sigma_base, log_prior_sigma_rel)

# Define prior predictive model and sample 1000 virtual patients
model = erlo.PriorPredictiveModel(predictive_model, log_prior)
n_samples = 1000
samples = model.sample(times, n_samples, seed)

# Visualise prior predictive model
fig = erlo.plots.PDPredictivePlot()
fig.add_prediction(data=samples, bulk_probs=[0.3, 0.6, 0.9])
fig.set_axis_labels(xlabel=r'$\text{Time in day}$', ylabel=r'$\text{Plasma conc. in ng/mL}$')
fig.show()

**Figure 2:** Prior predictive model of tumour volume measurement over time. The shaded areas estimate the 0.3, 0.6 and 0.9 bulk probability of the prior predictive distribution by sampling 1000 virtual "measurements" at each of the time points.

## Find MAP estimates for model parameters

In [26]:
# Set parameter names of mechanistic model (for convenience)
mechanistic_model.set_parameter_names(names={
        'central.drug_amount': 'Plasma drug amount in mg',
        'dose.drug_amount': 'Dose comp. drug amount in mg',
        'central.size': 'Volume of distribution in L',
        'dose.absorption_rate': 'Absorption rate in 1/d',
        'myokit.elimination_rate': 'Elimination rate in 1/d'})

# Create posterior
problem = erlo.ProblemModellingController(
    data, biom_keys=['Sample'])
problem.set_mechanistic_model(mechanistic_model)
problem.set_error_model(
    error_models=[pints.ConstantAndMultiplicativeGaussianLogLikelihood])
problem.fix_parameters(name_value_dict=dict({
    'Plasma drug amount in mg': 0,
    'Dose comp. drug amount in mg': 0,
    'Noise param 2': 1}))
problem.set_log_prior(log_priors=[
    log_prior_volume, 
    log_prior_absorption_rate,
    log_prior_elimination_rate,
    log_prior_sigma_base,
    log_prior_sigma_rel])
log_posterior = problem.get_log_posteriors()

# Find maximum a posteriori probability estimates (MAP)
opt = erlo.OptimisationController(log_posterior)
opt.set_transform(transform=pints.LogTransformation(n_parameters=5))
map_estimates = opt.run(show_run_progress_bar=True)

### Visualise optimisation results

In [27]:
fig = erlo.plots.ParameterEstimatePlot()
fig.add_data(map_estimates)

fig.show()

**Figure 3:** Maximum a posteriori (MAP) estimates of the model parameters. The y axis displays the estimated parameter value, and the x axis the corresponding individual.

## Find posterior probability distribution

In [28]:
# Set up sampling controller
sampler = erlo.SamplingController(log_posterior)
sampler.set_initial_parameters(data=map_estimates)
sampler.set_transform(transform=pints.LogTransformation(n_parameters=5))

# Run sampling
posterior_samples = sampler.run(n_iterations=4000, show_progress_bar=True)

### Visualise posterior samples

In [29]:
fig = erlo.plots.MarginalPosteriorPlot()
fig.add_data(data=posterior_samples, warm_up_iter=2000)

fig.show()

**Figure 4:** Marginal posterior distributions of model parameters. The y axis displays the sampled parameter value, and the x axis the binned number of samples for each individual.

## Posterior predictive checks

**WARNING:** This is currently not a posterior predictive check, but rather a check of the MAP estimates.

In [30]:
# Sample from predictive model
times = np.linspace(0, 1)
parameters = [0, 0, 0.74, 9.66, 3.26, 0.02, 0.06]
n_samples = 1000
samples = predictive_model.sample(parameters, times, n_samples)

# Visualise posterior predictive model
fig = erlo.plots.PDPredictivePlot()
fig.add_prediction(data=samples)
fig.add_data(data, biomarker='central.drug_concentration', meas_key='Sample')
fig.set_axis_labels(xlabel=r'$\text{Time in day}$', ylabel=r'$\text{Tumour volume in cm}^3$')
fig.show()

**Figure 5:** Predictive model constructed from the maximum a posteriori (MAP) estimates for the virtual patient. The shaded area illustrates the approximate 90% bulk probability of the predictive model constructed from 1000 samples for each of 50 equidistant time points, and the circles represent the measurements of the virtual patient.